In [39]:
import json
import os
import logging
import traceback
from pathlib import Path
from datetime import datetime
from typing import Optional

import torch
import tomli as tomlib
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm
from huggingface_hub import login
from llama_index.core import Settings, VectorStoreIndex, get_response_synthesizer, PromptTemplate
from llama_index.core.node_parser import TokenTextSplitter, SentenceSplitter
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.vector_store.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import SimilarityPostprocessor, NERPIINodePostprocessor, PrevNextNodePostprocessor
from llama_index.core.tools import QueryEngineTool
from llama_index.core.agent import AgentRunner, ReActAgentWorker
from llama_index.core.base.llms.types import ChatMessage
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
from llama_index.agent.openai import OpenAIAgentWorker
from trulens.core import TruSession
from trulens.core.feedback.feedback import Feedback
from trulens.providers.openai import OpenAI as OpenAIProvider
from trulens.providers.huggingface import Huggingface
from trulens.apps.llamaindex import TruLlama
from trulens.dashboard import run_dashboard

In [2]:
load_dotenv(override=True)

True

In [34]:
# parse configuration
with open('../pyproject.toml', "rb") as file:
    CFG = tomlib.load(file)

In [4]:
# Use your token here
login(token=CFG['configuration']['models']['hf_token'])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Data

In [5]:
# read dataset
with open(CFG['configuration']['data']['dataset_path'], 'rb') as file:
    dataset = json.load(file)

## Initialization

In [12]:
# connect to the existing neo4j vector index
vector_store = Neo4jVectorStore(
    username=CFG['configuration']['db']['username'],
    password=CFG['configuration']['db']['password'],
    url=CFG['configuration']['db']['url'],
    embedding_dimension=CFG['configuration']['embedding_dimension'],
    distance_strategy=CFG['configuration']['distance_strategy'],
    index_name=CFG['configuration']['db']['index_name'],
    text_node_property=CFG['configuration']['db']['text_node_property']
)

index = VectorStoreIndex.from_vector_store(vector_store)

### OpenAI

In [127]:
# initialize models
llm = OpenAI(
    api_key=os.getenv('AZURE_OPENAI_API_KEY'), 
    model=CFG['configuration']['models']['llm'],
    temperature=CFG['configuration']['models']['temperature']
)

embed_model = OpenAIEmbedding(
    model=CFG['configuration']['models']['embedding_model'],
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    dimensions=CFG['configuration']['embedding_dimension']
)

Settings.llm = llm
Settings.embed_model = embed_model

### Opensource

In [71]:
SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "<|start_header_id|>system<|end_header_id|>\n" + SYSTEM_PROMPT + "<|eot_id|><|start_header_id|>user<|end_header_id|>{query_str}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
)

llm = HuggingFaceLLM(
    context_window=CFG['configuration']['models']['context_window'],
    max_new_tokens=CFG['configuration']['models']['max_new_tokens'],
    generate_kwargs={"temperature": CFG['configuration']['models']['temperature'], "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=CFG['configuration']['models']['llm_hf'],
    model_name=CFG['configuration']['models']['llm_hf'],
    device_map="cuda:0",
    model_kwargs={"torch_dtype": torch.bfloat16}
)

Settings.llm = llm

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## QueryEngine

In [128]:
# create query engine
query_engine = index.as_query_engine()

In [129]:
# custom question
response = query_engine.query("What is the primary focus of Ukraine Boats Inc.?")

# get similarity scores
for node in response.source_nodes:
    print(f'{node.node.id_}, {node.score}')
    
# predicted answer
print(response.response)

ukraine-boats-3, 0.8536546230316162
ukraine-boats-4, 0.8363556861877441
The primary focus of Ukraine Boats Inc. is on designing and manufacturing boats that incorporate eco-friendly technologies, ensuring compliance with maritime safety standards, and expanding their market presence in Europe, North America, and Asia. They also emphasize customer relationships, quality control in production, and initiatives aimed at reducing their carbon footprint and making boating more accessible.


## Custom query engine

In [137]:
# custom retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=CFG['configuration']['similarity_top_k'],
    vector_store_query_mode=CFG['configuration']['vector_store_query_mode']
)

# similarity threshold
similarity_postprocessor = SimilarityPostprocessor(similarity_cutoff=CFG['configuration']['similarity_cutoff'])

# custom response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode=CFG['configuration']['response_mode']
)

# combine custom query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[similarity_postprocessor],
    response_synthesizer=response_synthesizer
)

In [138]:
# custom questions
response = query_engine.query("What is the primary focus of Ukraine Boats Inc.?")

In [139]:
for node in response.source_nodes:
    print(f'{node.node.id_}, {node.score}')

ukraine-boats-3, 0.8536546230316162
ukraine-boats-4, 0.8363556861877441
ukraine-boats-2, 0.8338797092437744
ukraine-boats-0, 0.8337738513946533
ukraine-boats-5, 0.83036208152771
ukraine-boats-6, 0.8298962116241455
ukraine-boats-7, 0.8119900226593018
ukraine-boats-9, 0.7885396480560303
ukraine-boats-1, 0.7823765277862549
ukraine-boats-10, 0.7761285305023193


In [140]:
response.response

'The primary focus of Ukraine Boats Inc. is to manufacture and supply high-quality boats and maritime solutions, specializing in recreational, commercial, and luxury vessels. The company emphasizes blending traditional craftsmanship with modern technology while integrating eco-friendly practices and ensuring compliance with maritime safety standards.'

## Agent

In [130]:
AGENT_SYSTEM_PROMPT = "You are a helpful human assistant. You always call the retrieve_semantically_similar_data tool before answering any questions. If the answer to the questions couldn't be found using the tool, just respond with `Didn't find relevant information`."
TOOL_NAME = "retrieve_semantically_similar_data"
TOOL_DESCRIPTION = "Provides additional information about the companies. Input: string"

### OpenAI

In [141]:
# agent worker
agent_worker = OpenAIAgentWorker.from_tools(
    [
        QueryEngineTool.from_defaults(
            query_engine=query_engine,
            name=TOOL_NAME,
            description=TOOL_DESCRIPTION,
            return_direct=False,
        )
    ],
    system_prompt=AGENT_SYSTEM_PROMPT,
    llm=llm,
    verbose=True,
    max_function_calls=CFG['configuration']['max_function_calls']
)

# agent runner
agent = AgentRunner(agent_worker=agent_worker)

### Opensource

In [104]:
agent_worker = ReActAgentWorker.from_tools(
    [
        QueryEngineTool.from_defaults(
            query_engine=query_engine,
            name=TOOL_NAME,
            description=TOOL_DESCRIPTION,
            return_direct=False,
        )
    ],
    llm=llm,
    verbose=True,
    chat_history=[ChatMessage(content=AGENT_SYSTEM_PROMPT, role="system")]
)

# agent runner
agent = AgentRunner(agent_worker=agent_worker)

## Chat

In [32]:
while True:
    # get user input
    current_message = input('Insert your next message:')
    print(f'{datetime.now().strftime("%H:%M:%S.%f")[:-3]}|User: {current_message}')

    response = agent.chat(current_message)
    print(f'{datetime.now().strftime("%H:%M:%S.%f")[:-3]}|Agent: {response.response}')

Insert your next message: Hi


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


16:04:29.117|User: Hi
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: retrieve_semantically_similar_data
Action Input: {'input': 'hello world', 'num_beams': 5}


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Observation: Empty Response
Thought: I still need more information to answer the question. The tool did not provide any useful output. I'll try to gather more context.
Action: retrieve_semantically_similar_data
Action Input: {'input': 'hello world', 'num_beams': 5}


...

Observation: 

CitySolve Municipal Services is a government-owned and operated company that serves as the backbone of New Urbania's civic infrastructure, addressing a wide range of city-level concerns.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: CitySolve Municipal Services is a government-owned and operated company that serves as the backbone of New Urbania's civic infrastructure, addressing a wide range of city-level concerns.
16:06:17.799|Agent: CitySolve Municipal Services is a government-owned and operated company that serves as the backbone of New Urbania's civic infrastructure, addressing a wide range of city-level concerns.


Insert your next message: Thanks


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


16:06:34.232|User: Thanks
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: CitySolve Municipal Services is a government-owned and operated company that serves as the backbone of New Urbania's civic infrastructure, addressing a wide range of city-level concerns.
16:06:35.734|Agent: CitySolve Municipal Services is a government-owned and operated company that serves as the backbone of New Urbania's civic infrastructure, addressing a wide range of city-level concerns.


KeyboardInterrupt: Interrupted by user

## Evaluation

In [44]:
tru = TruSession()
tru.reset_database()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [142]:
experiment_name = "gpt-4o-mini-custom-retriever-added-tool-inputs"

In [57]:
provider = OpenAIProvider(
    model_engine=CFG['configuration']['models']['llm_evaluation']
)

In [58]:
context_selection = TruLlama.select_source_nodes().node.text

# context relevance (for each of the context chunks)
f_context_relevance = (
    Feedback(
        provider.context_relevance, name="Context Relevance"
    )
    .on_input()
    .on(context_selection)
)

# groundedness
f_groundedness_cot = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context_selection.collect())
    .on_output()
)

# answer relevance between overall question and answer
f_qa_relevance = (
    Feedback(
        provider.relevance_with_cot_reasons, name="Answer Relevance"
    )
    .on_input_output()
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.calls[-1].rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [143]:
# Create TruLlama agent
tru_agent = TruLlama(
    agent,
    app_name=experiment_name,
    tags="agent testing",
    feedbacks=[f_qa_relevance, f_context_relevance, f_groundedness_cot],
)

In [144]:
tru_agent.print_instrumented()

Components:
	TruLlama (Custom) at 0x7f57b7bff700 with path __app__
	AgentRunner (Custom) at 0x7f57b6916090 with path __app__.app
	ChatMemoryBuffer (Custom) at 0x7f57b77ae4e0 with path __app__.app.memory
	SimpleChatStore (Custom) at 0x7f57b77ae170 with path __app__.app.memory.chat_store

Methods:
Object at 0x7f57b77ae4e0:
	<function BaseChatStoreMemory.put at 0x7f58a685ccc0> with path __app__.app.memory
	<function BaseMemory.put at 0x7f58a685c4a0> with path __app__.app.memory
Object at 0x7f57b6916090:
	<function AgentRunner.chat at 0x7f58a4fe8900> with path __app__.app
	<function AgentRunner.stream_chat at 0x7f58a4fe8680> with path __app__.app
	<function AgentRunner.achat at 0x7f58a4fe8f40> with path __app__.app
	<function AgentRunner.astream_chat at 0x7f58a4fe9300> with path __app__.app
	<function BaseQueryEngine.query at 0x7f58a6ce96c0> with path __app__.app
	<function BaseQueryEngine.aquery at 0x7f58a6ce9940> with path __app__.app
	<function BaseQueryEngine.synthesize at 0x7f58a6ce99

In [145]:
for item in tqdm(dataset):
    try:
        agent.reset()
        
        with tru_agent as recording:
            agent.query(item.get('question'))
        record_agent = recording.get()
        
        # wait until all the feedback function are finished
        for feedback, result in record_agent.wait_for_feedback_results().items():
            logging.info(f'{feedback.name}: {result.result}')
    except Exception as e:
        logging.error(e)
        traceback.format_exc()

  0% 0/10 [00:00<?, ?it/s]

Added user message to memory: What is the primary focus of Ukraine Boats Inc.?
=== Calling Function ===
Calling function: retrieve_semantically_similar_data with args: {"input":"Ukraine Boats Inc."}
Got output: Ukraine Boats Inc. is a premier manufacturer and supplier of high-quality boats and maritime solutions based in Odessa, Ukraine. Founded in 2005, the company specializes in recreational, commercial, and luxury vessels, blending traditional craftsmanship with modern technology. It has established a strong market presence in Europe, North America, and Asia, supported by partnerships with distribution companies like Baltic Marine Distributors in Germany, OceanCraft LLC in the USA, and Yokohama SeaTech in Japan.

The company is organized into several departments, including Engineering, Sales and Marketing, Production, and Customer Service, each with specific responsibilities to ensure efficient operations and customer satisfaction. Ukraine Boats Inc. is committed to sustainability t

 10% 1/10 [00:21<03:15, 21.76s/it]

Added user message to memory: What is the price range for recreational boats offered by Ukraine Boats Inc.?
=== Calling Function ===
Calling function: retrieve_semantically_similar_data with args: {"input":"price range for recreational boats offered by Ukraine Boats Inc."}
Got output: The price range for recreational boats offered is $32,000 to $55,000.



 20% 2/10 [00:39<02:33, 19.24s/it]

...

 80% 8/10 [02:36<00:34, 17.25s/it]

Added user message to memory: Which features make the WaveRunner X200 suitable for watersports?
=== Calling Function ===
Calling function: retrieve_semantically_similar_data with args: {"input":"WaveRunner X200 features for watersports"}
Got output: The WaveRunner X200 is designed for high-speed performance and durability in watersport racing. Key features include enhanced turbocharged engines and aerodynamic hull modifications, which contribute to its competitive edge on the water. These specifications enable teams to achieve superior speed and maneuverability, making it an ideal choice for dominating regional races.



 90% 9/10 [02:43<00:14, 14.20s/it]

Added user message to memory: What sustainability initiative is Ukraine Boats Inc. pursuing?
=== Calling Function ===
Calling function: retrieve_semantically_similar_data with args: {"input":"sustainability initiative Ukraine Boats Inc."}
Got output: The sustainability initiative at Ukraine Boats Inc. is the **Green Maritime Initiative (GMI)**, which aims to reduce the carbon footprint by incorporating renewable energy solutions in 50% of the fleet by 2030. Additionally, the company promotes the **SolarGlide EcoBoat**, a solar-powered vessel designed for environmentally conscious customers, featuring zero emissions and advanced solar technology.



100% 10/10 [02:58<00:00, 17.87s/it]


In [146]:
tru.get_leaderboard()

,,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_name,app_version,,,,,
gpt-4o-mini-custom-retriever,base,1.000000,0.588889,1.000000,6.702693,0.001112
gpt-4o-mini-custom-retriever-added-tool-inputs,base,1.000000,0.548889,1.000000,11.007236,0.001114
llama-3.2-3B-custom-retriever-added-tool-inputs,base,0.900000,0.616111,0.900000,6.842885,0.000000
gpt-4o-mini-default-retriever,base,0.800000,0.716667,0.793333,5.125279,0.000534
gpt-4o-mini-default-retriever-added-tool-inputs,base,0.800000,0.716667,0.793333,7.523859,0.000534
llama-3.2-3B-default-retriever-added-tool-inputs,base,0.700000,0.558333,0.833333,7.137055,0.000000
llama-3.2-3B-custom-retriever,base,0.541667,0.510251,0.708333,8.730055,0.000000
llama-3.2-3B-default-retriever,base,0.428571,0.523810,0.714286,8.830180,0.000000


In [ ]:
# tru.get_apps()
# tru.delete_app("app_hash_8f3f218c0136495a4835614dc1bbf1be")